# Stereo Camera Calibration Pipeline

This notebook performs complete stereo camera calibration including:
- Chessboard detection with parallel processing
- Robust RANSAC-based intrinsic calibration
- Stereo pair matching and diversity subsampling
- Stereo calibration and rectification
- Comprehensive visualizations

**Features:**
- Result caching for faster re-runs
- Configurable via YAML file
- Modular utilities for reuse in other scripts

## 1. Setup and Configuration

In [1]:
# Add utils to path
import random
import sys
from pathlib import Path

proj_root = Path.cwd().parent if Path.cwd().name == "calibration" else Path.cwd()
sys.path.insert(0, str(proj_root / "calibration"))

from utils import (
    CalibrationConfig,
    StereoRig,
    build_stereo_pairs_from_detections,
    calibrate_intrinsics_robust,
    calibrate_stereo,
    calibrate_stereo_robust,
    detect_chessboards_parallel,
    group_detections_by_image,
    load_calibration,
    load_kitti_calibration,
    plot_imagepoints_heatmap,
    plot_matched_boards,
    plot_rectification_preview,
    plot_stereo_pair_coverage,
    plot_undistortion_comparison,
    save_calibration,
    subsample_stereo_pairs,
    visualize_stereo_matches,
)

# Load configuration
config_path = proj_root / "calibration" / "config.yaml"
config = CalibrationConfig.from_yaml(config_path)
output_dir = proj_root / config.get("data.output_dir")
output_dir.mkdir(parents=True, exist_ok=True)
# Setup cache directory for faster re-runs
cache_dir = proj_root / config.get("data.cache_dir")
if cache_dir:
    cache_dir = cache_dir.resolve()
    print(f"Cache directory: {cache_dir}\n")

vis_mode = config.visualization_mode

print(f"Project root: {proj_root}")
print(f"Configuration loaded from: {config_path}")
print(f"Output folder (visualisations): {output_dir}")
print(f"Visualization mode: {vis_mode}")

Cache directory: C:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\cache

Project root: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project
Configuration loaded from: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\config.yaml
Output folder (visualisations): c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results
Visualization mode: save


## 2. Load Calibration Images

In [2]:
# Construct paths to calibration images
raw_data_folder = proj_root / config.get("data.raw_data_folder")
calib_left_dir = raw_data_folder / config.get("data.calib_left")
calib_right_dir = raw_data_folder / config.get("data.calib_right")

# Collect image paths
images_left = sorted(calib_left_dir.glob("*10.png"))
images_right = sorted(calib_right_dir.glob("*10.png"))

print(f"Found {len(images_left)} left images in: {calib_left_dir}")
print(f"Found {len(images_right)} right images in: {calib_right_dir}")

assert len(images_left) > 0 and len(images_right) > 0, "No calibration images found!"

Found 1 left images in: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\34759_final_project_raw\calib\image_02\data
Found 1 right images in: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\34759_final_project_raw\calib\image_03\data


## 3. Detect Chessboards

Detect all chessboard patterns in both left and right calibration images using parallel processing.

In [3]:
pattern_sizes = config.pattern_sizes
max_boards = config.get("detection.max_boards_per_image", 13)
num_workers = config.get("detection.num_workers", -1)

print(f"Detecting chessboards with patterns: {pattern_sizes}")
print(f"Using {num_workers if num_workers > 0 else 'all'} CPU cores\n")

# Detect LEFT
print("=== LEFT CAMERA ===")
corners_left, objpoints_left, indices_left, image_size = detect_chessboards_parallel(
    [str(p) for p in images_left],
    pattern_sizes,
    max_boards=max_boards,
    num_workers=num_workers,
    progress=True,
    cache_dir=cache_dir,
)
print(f"Detected {len(corners_left)} boards in {len(images_left)} images\n")

# Detect RIGHT
print("=== RIGHT CAMERA ===")
corners_right, objpoints_right, indices_right, _ = detect_chessboards_parallel(
    [str(p) for p in images_right],
    pattern_sizes,
    max_boards=max_boards,
    num_workers=num_workers,
    progress=True,
    cache_dir=cache_dir,
)
print(f"Detected {len(corners_right)} boards in {len(images_right)} images")
print(f"\nImage size: {image_size}")

Detecting chessboards with patterns: [(11, 7), (15, 5), (7, 5)]
Using all CPU cores

=== LEFT CAMERA ===


Detecting boards:   0%|          | 0/1 [00:00<?, ?it/s]

Cached detection results (key=3d351614...)
Detected 13 boards in 1 images

=== RIGHT CAMERA ===


Detecting boards:   0%|          | 0/1 [00:00<?, ?it/s]

Cached detection results (key=23ea429f...)
Detected 11 boards in 1 images

Image size: (1392, 512)


## 4. Robust Intrinsic Calibration

Perform robust RANSAC-based intrinsic calibration for each camera individually.

In [4]:
# Initialize stereo rig
rig = StereoRig()

# Populate detections
rig.left.objpoints = objpoints_left
rig.left.imgpoints = corners_left
rig.left.image_indices = indices_left
rig.left.image_size = image_size

rig.right.objpoints = objpoints_right
rig.right.imgpoints = corners_right
rig.right.image_indices = indices_right
rig.right.image_size = image_size

print("Stereo rig initialized with detection data")

Stereo rig initialized with detection data


In [5]:
# Load robust calibration parameters from config
robust_params = config.config.get("robust_calibration", {})

# Calibrate LEFT camera
print("=" * 60)
print("LEFT CAMERA ROBUST CALIBRATION")
print("=" * 60)

result_left = calibrate_intrinsics_robust(
    rig.left.objpoints,
    rig.left.imgpoints,
    image_size,
    image_indices_list=rig.left.image_indices,
    cache_dir=cache_dir,
    **robust_params,
)

rig.left.K = result_left.K
rig.left.dist = result_left.dist

print("\n" + result_left.summary())

LEFT CAMERA ROBUST CALIBRATION
[Stage 1] Naive calibration on up to 13 boards (removed stage0=0/13, geom duplicates=0)


Naive RMS:   0%|          | 0/13 [00:00<?, ?it/s]

[Stage 3] RANSAC search
RANSAC: iters=200 subset_size=6 candidates=13


RANSAC:   0%|          | 0/200 [00:00<?, ?it/s]

RANSAC best: inliers=13/13 RMS=0.224
[Stage 4] Post-rejection tightening


Post RMS:   0%|          | 0/13 [00:00<?, ?it/s]

Post-rejection: thr=0.304 removed=1
[Stage 5] Final calibration

Robust Calibration Summary:
  Total detections         : 13
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 0
  Diversity selected       : 13
  Removed pre-rejection    : 0
  Removed post-rejection   : 1
  Final boards             : 12
  RMS naive                : 0.1944 px
  RMS final                : 0.1953 px
  Improvement              : -0.0009 px
  Runtime                  : 27.75 s


Robust Calibration Summary:
  Total detections         : 13
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 0
  Diversity selected       : 13
  Removed pre-rejection    : 0
  Removed post-rejection   : 1
  Final boards             : 12
  RMS naive                : 0.1944 px
  RMS final                : 0.1953 px
  Improvement              : -0.0009 px
  Runtime                  : 27.75 s



In [6]:
# Calibrate RIGHT camera
print("\n" + "=" * 60)
print("RIGHT CAMERA ROBUST CALIBRATION")
print("=" * 60)

result_right = calibrate_intrinsics_robust(
    rig.right.objpoints,
    rig.right.imgpoints,
    image_size,
    image_indices_list=rig.right.image_indices,
    cache_dir=cache_dir,
    **robust_params,
)

rig.right.K = result_right.K
rig.right.dist = result_right.dist

print("\n" + result_right.summary())


RIGHT CAMERA ROBUST CALIBRATION
[Stage 1] Naive calibration on up to 11 boards (removed stage0=0/11, geom duplicates=0)
[Stage 3] RANSAC search
RANSAC: iters=200 subset_size=6 candidates=11
[Stage 3] RANSAC search
RANSAC: iters=200 subset_size=6 candidates=11


RANSAC:   0%|          | 0/200 [00:00<?, ?it/s]

RANSAC best: inliers=11/11 RMS=0.116
[Stage 4] Post-rejection tightening
Post-rejection: thr=0.169 removed=1
[Stage 5] Final calibration

Robust Calibration Summary:
  Total detections         : 11
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 0
  Diversity selected       : 11
  Removed pre-rejection    : 0
  Removed post-rejection   : 1
  Final boards             : 10
  RMS naive                : 0.0901 px
  RMS final                : 0.0817 px
  Improvement              : 0.0084 px
  Runtime                  : 28.17 s


Robust Calibration Summary:
  Total detections         : 11
  Removed stage0           : 0
  Removed duplicates (geom): 0
  Removed duplicates (extr): 0
  Diversity selected       : 11
  Removed pre-rejection    : 0
  Removed post-rejection   : 1
  Final boards             : 10
  RMS naive                : 0.0901 px
  RMS final                : 0.0817 px
  Improvement              : 0.0084 px
  Runtime                  : 28

## 5. Visualize Intrinsic Calibration Results

In [7]:
# Undistortion comparison - LEFT
sample_left = str(random.choice(images_left))
plot_undistortion_comparison(
    sample_left,
    rig.left.K,
    rig.left.dist,
    camera_name="LEFT",
    output_path=output_dir / "undistortion_left.png",
    mode=vis_mode,
)

# Undistortion comparison - RIGHT
sample_right = str(random.choice(images_right))
plot_undistortion_comparison(
    sample_right,
    rig.right.K,
    rig.right.dist,
    camera_name="RIGHT",
    output_path=output_dir / "undistortion_right.png",
    mode=vis_mode,
)

In [8]:
# Coverage heatmaps
heatmap_bins = config.get("visualization.heatmap_bins", 40)

plot_imagepoints_heatmap(
    result_left.final_imgpoints,
    image_size,
    camera_name="LEFT",
    output_path=output_dir / "heatmap_left.png",
    mode=vis_mode,
    bins=heatmap_bins,
)

plot_imagepoints_heatmap(
    result_right.final_imgpoints,
    image_size,
    camera_name="RIGHT",
    output_path=output_dir / "heatmap_right.png",
    mode=vis_mode,
    bins=heatmap_bins,
)


LEFT Coverage Statistics:
  Total points: 544
  Total boards: 12
  X range: [163.7, 1373.2] (coverage: 86.9%)
  Y range: [84.4, 465.0] (coverage: 74.3%)

RIGHT Coverage Statistics:
  Total points: 392
  Total boards: 10
  X range: [98.4, 1249.1] (coverage: 82.7%)
  Y range: [82.7, 453.6] (coverage: 72.4%)

RIGHT Coverage Statistics:
  Total points: 392
  Total boards: 10
  X range: [98.4, 1249.1] (coverage: 82.7%)
  Y range: [82.7, 453.6] (coverage: 72.4%)


## 6. Build Stereo Pairs from Detected Boards

Match chessboards between left and right images to create stereo calibration pairs. This uses the already-detected boards from section 3.

In [9]:
# Build stereo pairs by matching already-detected boards
matching_params = config.config.get("matching", {})
square_size = config.square_size

stereo_objpoints, stereo_imgpoints_left, stereo_imgpoints_right, matching_metadata = build_stereo_pairs_from_detections(
    corners_left,
    objpoints_left,
    indices_left,
    corners_right,
    objpoints_right,
    indices_right,
    dy_thresh=matching_params.get("dy_thresh", 12.0),
    scale_ratio_thresh=matching_params.get("scale_ratio_thresh", 0.25),
    method=matching_params.get("method", "hungarian"),
    cache_dir=cache_dir,
    progress=True,
)

# Scale object points by square size
for objp in stereo_objpoints:
    objp *= float(square_size)

print("\nMatching statistics:")
print(f"  Total pairs processed: {matching_metadata['total_pairs_processed']}")
print(f"  Total boards matched: {matching_metadata['total_boards_matched']}")
if matching_metadata["matches_per_pair"]:
    print(
        f"  Average matches per pair: {sum(matching_metadata['matches_per_pair']) / len(matching_metadata['matches_per_pair']):.1f}"
    )

# Visualize a few example matched pairs
visualize_stereo_matches(
    corners_left,
    objpoints_left,
    indices_left,
    corners_right,
    objpoints_right,
    indices_right,
    images_left,
    images_right,
    matching_metadata,
    output_dir,
    vis_mode=vis_mode,
    max_pairs=3,
)

Matching chessboards across 1 stereo pairs...

Pair 0: 11 matched boards (total so far: 11)

Total matched boards across all pairs: 11
Cached stereo_matching results (key=9ebaae1c...)

Matching statistics:
  Total pairs processed: 1
  Total boards matched: 11
  Average matches per pair: 11.0

Visualizing matches for pairs: [0]
Pair 0: 11 matched boards (total so far: 11)

Total matched boards across all pairs: 11
Cached stereo_matching results (key=9ebaae1c...)

Matching statistics:
  Total pairs processed: 1
  Total boards matched: 11
  Average matches per pair: 11.0

Visualizing matches for pairs: [0]


## 6b. Subsample Stereo Pairs for Diversity

Apply diversity-based subsampling to stereo pairs to reduce redundancy and speed up stereo calibration while maintaining good geometric coverage.

In [10]:
# Subsample stereo pairs for diversity
subsample_config = config.config.get("stereo_subsampling", {})
if subsample_config.get("enabled", True):
    print("\n" + "=" * 60)
    print("STEREO PAIR DIVERSITY SUBSAMPLING")
    print("=" * 60)

    stereo_objpoints_subs, stereo_imgpoints_subs_left, stereo_imgpoints_subs_right, kept_indices = (
        subsample_stereo_pairs(
            stereo_objpoints,
            stereo_imgpoints_left,
            stereo_imgpoints_right,
            rig.left.K,
            rig.left.dist,
            max_pairs=subsample_config.get("max_pairs", 100),
            min_angle_deg=subsample_config.get("min_angle_deg", 10.0),
            min_translation=subsample_config.get("min_translation", 0.05),
            verbose=True,
        )
    )

    # print(f"\nKept {len(kept_indices)} pairs from original {total_matched}")

    # Visualize spatial coverage of stereo pairs
    plot_stereo_pair_coverage(
        stereo_imgpoints_left,
        stereo_imgpoints_right,
        image_size,
        kept_indices=kept_indices,
        output_path=output_dir / "stereo_pair_coverage.png",
        mode=vis_mode,
    )
    print("Stereo pair coverage visualization saved")
else:
    stereo_objpoints_subs = stereo_objpoints
    stereo_imgpoints_subs_left = stereo_imgpoints_left
    stereo_imgpoints_subs_right = stereo_imgpoints_right
    print("\nStereo pair subsampling disabled, using all matched pairs")


STEREO PAIR DIVERSITY SUBSAMPLING
Stereo pairs (11) <= max_pairs (100), no subsampling needed
Stereo pair coverage visualization saved
Stereo pair coverage visualization saved


## 7. Stereo Calibration and Rectification

In [11]:
rig.left.K, rig.left.dist, rig.right.K, rig.right.dist

(array([[1.15564114e+03, 0.00000000e+00, 6.90691526e+02],
        [0.00000000e+00, 1.04950315e+03, 2.55781555e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[-1.93258364e-01, -3.81307206e-01,  4.70666348e-04,
         -1.06196264e-02,  5.54117285e-01]]),
 array([[962.80490504,   0.        , 703.08434391],
        [  0.        , 966.6622743 , 255.49797635],
        [  0.        ,   0.        ,   1.        ]]),
 array([[-4.20494116e-01,  4.34087472e-01,  6.17274600e-05,
         -2.34462392e-03, -2.96691157e-01]]))

In [12]:
# Stereo calibration parameters
stereo_params = config.config.get("stereo", {})

print("\n" + "=" * 60)
print("STEREO CALIBRATION")
print("=" * 60 + "\n")

# Use robust RANSAC-like calibration
rig, best_indices = calibrate_stereo_robust(
    rig,
    stereo_objpoints_subs,
    stereo_imgpoints_subs_left,
    stereo_imgpoints_subs_right,
    image_size,
    ransac_iterations=100,
    subset_size=min(6, len(stereo_objpoints_subs)),
    **stereo_params,
)

print("\nStereo calibration complete!")
print(f"  RMS error: {rig.stereo_rms:.4f} px")
print(f"  Baseline: {rig.baseline:.4f} m")
print(f"  Left ROI: {rig.left.roi}")
print(f"  Right ROI: {rig.right.roi}")
print(f"  Best subset size: {len(best_indices)}")


STEREO CALIBRATION

Robust Stereo Calibration: 11 pairs, 100 iters, subset=6
  Iter 0: FULL -> RMS=31.2646 px (Train RMS=29.9290)
  Iter 1: SUB(6) -> RMS=31.4527 px (Train RMS=31.9912)
  Iter 2: SUB(6) -> RMS=33.4319 px (Train RMS=27.0299)
  Iter 0: FULL -> RMS=31.2646 px (Train RMS=29.9290)
  Iter 1: SUB(6) -> RMS=31.4527 px (Train RMS=31.9912)
  Iter 2: SUB(6) -> RMS=33.4319 px (Train RMS=27.0299)
  Iter 3: SUB(6) -> RMS=33.5989 px (Train RMS=26.1153)
  Iter 4: SUB(6) -> RMS=35.3495 px (Train RMS=30.9907)
  Iter 3: SUB(6) -> RMS=33.5989 px (Train RMS=26.1153)
  Iter 4: SUB(6) -> RMS=35.3495 px (Train RMS=30.9907)
  Iter 5: SUB(6) -> RMS=34.0023 px (Train RMS=31.8120)
  Iter 6: SUB(6) -> RMS=35.5282 px (Train RMS=30.1727)
  Iter 5: SUB(6) -> RMS=34.0023 px (Train RMS=31.8120)
  Iter 6: SUB(6) -> RMS=35.5282 px (Train RMS=30.1727)
  Iter 7: SUB(6) -> RMS=32.4520 px (Train RMS=32.8583)
  Iter 8: SUB(6) -> RMS=35.0500 px (Train RMS=27.8284)
  Iter 7: SUB(6) -> RMS=32.4520 px (Train RMS=

## 8. Visualize Rectification Results

In [13]:
# Pick a random stereo pair for visualization
test_idx = random.randint(0, len(images_left) - 1)
n_guides = config.get("visualization.rectification_guide_lines", 12)

plot_rectification_preview(
    str(images_left[test_idx]),
    str(images_right[test_idx]),
    rig.left.map1,
    rig.left.map2,
    rig.right.map1,
    rig.right.map2,
    roi_left=rig.left.roi,
    roi_right=rig.right.roi,
    n_guides=n_guides,
    output_path=output_dir / f"rectification_preview_{test_idx}.png",
    mode=vis_mode,
)

print(f"Rectification preview saved for image pair {test_idx}")

Rectification preview saved for image pair 0


## 9. Export Calibration Parameters

In [14]:
# Export configuration
export_format = config.get("export.format", "yaml")
output_filename = config.get("export.output_filename", "stereo_calibration.yaml")
output_path = output_dir / output_filename

save_calibration(
    rig,
    output_path,
    fmt=export_format,
    include_metadata=config.get("export.include_metadata", True),
)

print(f"\nCalibration results exported to: {output_path}")
print(f"Format: {export_format}")

Calibration saved to c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml

Calibration results exported to: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml
Format: yaml


## 10. Verify Saved Calibration (Optional)

Load the saved calibration back and verify parameters.

In [15]:
# Load and verify
rig_loaded = load_calibration(output_path)

print("\nVerification:")
print(f"  Left camera calibrated: {rig_loaded.left.is_calibrated()}")
print(f"  Right camera calibrated: {rig_loaded.right.is_calibrated()}")
print(f"  Stereo calibrated: {rig_loaded.is_stereo_calibrated()}")
print(f"  Rectified: {rig_loaded.is_rectified()}")
print(f"  Baseline: {rig_loaded.baseline:.4f} m")

# Compare key parameters
import numpy as np

k_diff = np.linalg.norm(rig.left.K - rig_loaded.left.K)
print(f"  K matrix difference (L2 norm): {k_diff:.2e}")

Calibration loaded from c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\calibration\results\stereo_calibration.yaml

Verification:
  Left camera calibrated: True
  Right camera calibrated: True
  Stereo calibrated: True
  Rectified: False
  Baseline: 1.5140 m
  K matrix difference (L2 norm): 0.00e+00


## Summary

Calibration pipeline complete! The following files have been generated:

- Calibration parameters: `{output_dir}/{output_filename}`
- Visualization outputs: `{output_dir}/*.png`

Use `load_calibration()` to load the parameters in other scripts.

## 11. Compare with Provided Calibration

Compare the calculated calibration parameters with the provided `calib_cam_to_cam.txt` file.

In [16]:
import numpy as np

# Path to provided calibration file
provided_calib_path = proj_root / "data" / "calib_cam_to_cam.txt"

if provided_calib_path.exists():
    print(f"Loading provided calibration from: {provided_calib_path}")
    rig_provided = load_kitti_calibration(provided_calib_path)

    print("\n" + "=" * 60)
    print("CALIBRATION COMPARISON (Calculated vs Provided)")
    print("=" * 60)

    def compare_matrix(name, mat_calc, mat_prov):
        if mat_calc is None or mat_prov is None:
            print(f"\n{name}: Missing data for comparison")
            return

        diff = np.abs(mat_calc - mat_prov)
        abs_err = np.mean(diff)

        # Avoid division by zero
        denom = np.abs(mat_prov)
        denom[denom < 1e-9] = 1.0
        perc_err = np.mean(diff / denom) * 100.0

        print(f"\n{name}:")
        print(f"  Mean Abs Diff: {abs_err:.6f}")
        print(f"  Mean % Diff:   {perc_err:.4f}%")
        print(f"  Max Diff:      {np.max(diff):.6f}")

    # Compare Intrinsics
    print("\n--- LEFT CAMERA ---")
    compare_matrix("K Matrix", rig.left.K, rig_provided.left.K)
    compare_matrix("Distortion", rig.left.dist.flatten()[:5], rig_provided.left.dist.flatten())

    print("\n--- RIGHT CAMERA ---")
    compare_matrix("K Matrix", rig.right.K, rig_provided.right.K)
    compare_matrix("Distortion", rig.right.dist.flatten()[:5], rig_provided.right.dist.flatten())

    # Compare Stereo
    print("\n--- STEREO EXTRINSICS ---")
    if rig.baseline is not None and rig_provided.baseline is not None:
        b_calc = rig.baseline
        b_prov = rig_provided.baseline
        diff = abs(b_calc - b_prov)
        perc = (diff / b_prov) * 100.0
        print("\nBaseline:")
        print(f"  Calculated: {b_calc:.6f} m")
        print(f"  Provided:   {b_prov:.6f} m")
        print(f"  Diff:       {diff:.6f} m ({perc:.4f}%)")

    # Compare Translation Vector
    # Note: rig.T is usually [ -baseline, 0, 0 ] for rectified, but raw T might differ
    # rig_provided.T was calculated from T_03 - T_02
    compare_matrix("Translation (T)", rig.T, rig_provided.T)

else:
    print(f"Provided calibration file not found at: {provided_calib_path}")


Loading provided calibration from: c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\calib_cam_to_cam.txt
KITTI calibration loaded from c:\Users\fabiu\Documents\GitHub\34759_PfAS_project\data\calib_cam_to_cam.txt

CALIBRATION COMPARISON (Calculated vs Provided)

--- LEFT CAMERA ---

K Matrix:
  Mean Abs Diff: 35.157803
  Mean % Diff:   4.2943%
  Max Diff:      198.693642

Distortion:
  Mean Abs Diff: 0.282379
  Mean % Diff:   388.9192%
  Max Diff:      0.628810

--- RIGHT CAMERA ---

K Matrix:
  Mean Abs Diff: 17.093944
  Mean % Diff:   2.5288%
  Max Diff:      69.598374

Distortion:
  Mean Abs Diff: 0.106417
  Mean % Diff:   285.3356%
  Max Diff:      0.243221

--- STEREO EXTRINSICS ---

Baseline:
  Calculated: 1.514009 m
  Provided:   0.535660 m
  Diff:       0.978349 m (182.6436%)

Translation (T):
  Mean Abs Diff: 0.626330
  Mean % Diff:   5069.4457%
  Max Diff:      1.494144
